In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import neurokit2 as nk
import pyhrv.frequency_domain as fd
import pyhrv.time_domain as td

from ipynb.fs.full.get_samples import get_data, get_samples_signal
from ipynb.fs.full.ECG_features import filter_ecgsignal

In [ ]:
fs =700
subject ='S2'
label = 2
data = get_data(subject)
labels = data.get_labels()
ecg, resp, eda_base = get_samples_signal(data, labels, label, fs)

filtered_ecg = filter_ecgsignal(ecg, fs)
rpeaks, info = nk.ecg_peaks(ecg, sampling_rate=fs)
ecg_rate = nk.ecg_rate(rpeaks, sampling_rate=fs, desired_length=len(ecg))
edr = nk.ecg_rsp(ecg_rate, sampling_rate=fs)
resp_peaks1, _ = signal.find_peaks(edr, height=0, distance=4)
nk.signal_plot(edr)

In [ ]:
from scipy.interpolate import splrep, splev

ecg_td = td.ecg(ecg, interactive= False, show = False, sampling_rate=700)
rr = ecg_td['rpeaks']
filtered_ecg = ecg_td['filtered']

fs = 700
# rr = rpeaks
rr = (rr / fs) * 1000
rri = np.diff(rr)
print(rri)

def interp_cubic_spline(rri, sf_up=4):
    rri_time = np.cumsum(rri) / 1000.0
    time_rri = rri_time - rri_time[0]
    time_rri_interp = np.arange(0, time_rri[-1], 1 / float(sf_up))
    tck = splrep(time_rri, rri, s=0)
    rri_interp = splev(time_rri_interp, tck, der=0)
    return rri_interp

sf_up = 4
rri_interp = interp_cubic_spline(rri, sf_up) 
hr = 1000 * (60 / rri_interp)
print('Mean HR: %.2f bpm' % np.mean(hr))

# Detrend and normalize
edr = signal.detrend(hr)
edr = (edr - edr.mean()) / edr.std()

# Find respiratory peaks
resp_peaks, _ = signal.find_peaks(edr, height=0, distance=sf_up)

# Convert to seconds
resp_peaks = resp_peaks
resp_peaks_diff = np.diff(resp_peaks) / sf_up

# Plot the EDR waveform
plt.plot(edr)
plt.plot(resp_peaks, edr[resp_peaks], 'o')
_ = plt.title('ECG derived respiration')

In [ ]:
# rpeaks, info = nk.ecg_peaks(resp, sampling_rate=fs)
resp_peaks2, _ = signal.find_peaks(resp, height=2, distance=30)
maxi = 0
begini = 0
peaks = []

for i in range(len(resp_peaks2)):
    if resp_peaks2[i] >= resp_peaks2[begini] + 400:
        begini = i
        peaks.append(resp_peaks2[maxi])
        maxi = i

    if resp[resp_peaks2[i]]> resp[resp_peaks2[maxi]]:
        maxi = i

print(peaks)
plt.figure(figsize=(8,6))
plt.plot(resp)
plt.plot(peaks, resp[peaks], 'o')

In [ ]:
print(len(resp_peaks1),len(resp_peaks2))
print(peaks)
print(resp_peaks1, resp_peaks)